### Fencing matches - preliminary analysis

In [ ]:
%load_ext autoreload
    
%autoreload 2

In [ ]:
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets


%matplotlib inline

In [ ]:
df = pd.read_csv('./data/Kaggle/ClipDescriptorKaggle.csv')

In [ ]:
df.head()

In [ ]:
df.nunique(axis=0)

In [ ]:
df[df['URL'].str.contains('uho2ldovj7o')]

In [ ]:

def load_key_point(path):
    
    with open(path, 'r') as fh:
        df_kp = json.load(fh)
    res = [np.array(el) for el in df_kp]
    return [el for el in res if el.ndim == 3]

df_kp = load_key_point('./data/Kaggle/FencersKeyPoints/0001_002_00_02_1.json')

In [ ]:
df_kp[0].shape

In [ ]:
from ipywidgets import interactive

from fencing.openpose import POSE_LABELS


def plot_key_points(X, ax, annotate=False):
    mask = X[0].mean(axis=1) > 1
    ax.plot(X[0][mask, 0], X[0][mask, 1], 'ro')
    if annotate:
        for idx in np.nonzero(mask)[0]:
            ax.text(X[0][idx, 0], X[0][idx, 1], s=str(idx))
    mask = X[1].mean(axis=1) > 1
    ax.plot(X[1][mask, 0], X[1][mask, 1], 'bo')
    if annotate:
        for idx in np.nonzero(mask)[0]:
            ax.text(X[1][idx, 0], X[1][idx, 1], s=str(idx))
    

    ax.set_ylim(280, 0)
    
def plot_key_point_sequence(idx):
    fig, ax = plt.subplots(figsize=(8, 6))
    plot_key_points(df_kp[int(idx)], ax, annotate=True)
    plt.show()

#

interactive_plot = interactive(plot_key_point_sequence, idx=(0, len(df_kp)-1))
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

In [ ]:
len(df_kp)

### Feature extraction

In [ ]:
from fencing.feature_extraction import extract_features


df_fe = extract_features(df_kp)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 10))
#x = np.linspace(0, 5, len(leg_dist))
df_fe.plot(y="leg_distance_p0", color='r', ax=ax[0])
df_fe.plot(y="leg_distance_p1", color='b', ax=ax[0])
#df_fe.plot(y="right_leg_angle_p0", color='r', ax=ax[1])
#df_fe.plot(y="right_leg_angle_p1", color='b', ax=ax[1])
df_fe.plot(y="mhip_distance_x", color='k', ax=ax[1])
df_fe.plot(y="mhip_speed_x_diff", ls='dashed', color='k', ax=ax[2])
df_fe.plot(y="mhip_speed_x_p0", color='r', ax=ax[2])
df_fe.plot(y="mhip_speed_x_p1", color='b', ax=ax[2])
ax[0].legend()